# Índice Jerárquico

Hasta ahora, nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas `` Series `` y `` DataFrame ``, respectivamente.

A menudo, es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.

Si bien Pandas proporciona objetos `` Panel `` y `` Panel4D `` (que mencionaremos más adelante) que manejan de forma nativa datos tridimensionales y tetradimensionales, un método mucho más común en la práctica es hacer uso de la *indexación jerárquica* (también conocida como *indexación múltiple*) para incorporar múltiples índices (o niveles) dentro de un solo índice.

De esta manera, los datos de mayor dimensión se pueden representar de forma compacta dentro de los objetos familiares unidimensionales `` Series `` y bidimensionales `` DataFrame ``.

En esta sección, exploraremos la creación directa de objetos `` MultiIndex ``, consideraciones al indexar, dividir y calcular estadísticas a través de datos indexados múltiples, y rutinas útiles para realizar conversiones entre representaciones simples e indexadas jerárquicamente.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

## Series con índices múltiples

Comencemos por considerar cómo podríamos representar datos bidimensionales dentro de una `` Serie `` unidimensional.
Para mayor concreción, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### El camino que deberías evitar

Supongamos que nos gustaría rastrear datos sobre los estados de USA para dos años diferentes.
Con las herramientas de Pandas que ya hemos cubierto, es posible que tengamos la tentación de usar simplemente tuplas de Python como claves:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Con este esquema de indexación, podemos indexar o dividir directamente la serie en función de este índice múltiple:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pero la conveniencia termina ahí. Por ejemplo, si necesitásemos seleccionar todos los valores de 2010, necesitaríamos hacer algunas tareas desordenadas (y lentas) para conseguirlo:

In [11]:
l = []

for i in pop.index:
    if i[1] == 2010:
        l.append(i)
        
l

[('California', 2010), ('New York', 2010), ('Texas', 2010)]

In [12]:
# pop[l]
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Con esto conseguimos el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de segmentación que nos proporciona Pandas.

### Un camino mejor: Pandas MultiIndex

Afortunadamente, Pandas ofrece una manera mucho mejor, que será más eficiente y nos evitará posibles quebraderos de cabeza.

Nuestra indexación basada en tuplas es esencialmente un índice múltiple rudimentario, y el tipo Pandas `` MultiIndex `` nos ofrece el tipo de operaciones que deseamos tener.

Podemos crear un MultiIndex a partir de las tuplas de la siguiente manera:

In [13]:
index

[('California', 2000),
 ('California', 2010),
 ('New York', 2000),
 ('New York', 2010),
 ('Texas', 2000),
 ('Texas', 2010)]

In [14]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Fíjate que el `` MultiIndex `` contiene múltiples niveles de indexación. En este caso, los nombres de los estados y los años. Pero el nivel no termina ahí, sino que podríamos enlazar más niveles.

Si volvemos a indexar nuestra serie con este `` MultiIndex ``, podremos ver la representación jerárquica de los datos:

In [15]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Aquí, las dos primeras columnas de la representación de la `` Serie `` muestran los valores del MultiIndex, mientras que la tercera columna muestra los datos. 
Fíjate que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea que está encima.

Ahora, para acceder a todos los datos para los que el segundo índice es 2010, simplemente podemos usar la indexación de Pandas que hemos viso hasta ahora:

In [20]:
pop['California']

2000    33871648
2010    37253956
dtype: int64

El resultado es una matriz indexada individualmente con solo las claves que nos interesan.

Esta sintaxis es mucho más apropiada (además de eficiente) que la solución de indexación múltiple basada en tuplas caseras que hemos visto antes.

A continuación, analizaremos más a fondo este tipo de operación de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Si pensamos un poco, podríamos haber almacenado fácilmente los mismos datos utilizando un simple `` DataFrame `` con sus etiquetas de índice y columna.

De hecho, Pandas se construye con esta equivalencia en mente. El método `unstack ()` convertirá rápidamente una `Serie` indexada de forma múltiple en un ``DataFrame`` indexado convencionalmente:

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


De forma análoga, el método ``stack()`` hace exactamente lo contrario:

In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Al ver esto, es posible preguntarse por qué utilizar la indexación jerárquica.

La razón es simple: así como pudimos usar la indexación múltiple para representar datos bidimensionales dentro de una `` Serie `` unidimensional, también podemos usarla para representar datos de tres o más dimensiones en una `` Serie `` o `` DataFrame ``.

Cada nivel adicional en un índice múltiple representa una dimensión adicional de datos; aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer agregar otra columna de datos demográficos para cada estado de cada año (por ejemplo, población menor de 18 años). Con un `` MultiIndex `` esto es tan sencillo como agregar otra columna al `` DataFrame ``:

In [22]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Además, todas las ufuncs que hemos visto, así como otras funcionalidades comentadas, también funcionan con los índices jerárquicos.

A continuación, calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [25]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


Esto nos permite manipular y realizar una exploración de manera rápida y fácil, incluso con datos de alta dimensión.

## Métodos de creación con MultiIndex

La forma más sencilla de construir una `` Serie `` o `` DataFrame `` indexados de forma múltiple es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo:

In [26]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.972308  0.567182
  2  0.989949  0.864869
b 1  0.574606  0.186551
  2  0.921316  0.683423

El trabajo de crear el `` MultiIndex `` se realiza en segundo plano.

De manera similar, si pasamos un diccionario con las tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y usará un `` MultiIndex `` por defecto:

In [27]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

Sin embargo, a veces es útil crear explícitamente un `` MultiIndex ``. Veamos un par de estos métodos:

### Constructores explícitos MultiIndex

Para mayor flexibilidad en la forma en que se construye el índice, en su lugar puede utilizar los constructores de métodos de clase disponibles en el objeto `` pd.MultiIndex ``.

Por ejemplo, como hemos hecho antes, podríamos construir el `` MultiIndex `` a partir de una lista simple de matrices que dan los valores de índice dentro de cada nivel:

In [30]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Podemos construirlo a partir de una lista de tuplas, especificando cada posible valor de la tupla para cada punto:

In [32]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Podríamos incluso construirlo a partir del producto cartesiano de los índices únicos:

In [33]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2], ['I', 'II', 'III']])

MultiIndex([('a', 1,   'I'),
            ('a', 1,  'II'),
            ('a', 1, 'III'),
            ('a', 2,   'I'),
            ('a', 2,  'II'),
            ('a', 2, 'III'),
            ('b', 1,   'I'),
            ('b', 1,  'II'),
            ('b', 1, 'III'),
            ('b', 2,   'I'),
            ('b', 2,  'II'),
            ('b', 2, 'III')],
           )

De manera similar, podríamos construir un ``MultiIndex`` directamente usando su codificación interna pasándole los ``levels`` (una lista de listas que contienen valores de índice disponibles para cada nivel) y `` codes `` (una lista de listas que hacen referencia a estas etiquetas):

In [38]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Cualquiera de estos objetos se puede pasar como el argumento "índice" al crear una ``Serie`` o un ``DataFrame``, o se puede pasar al método `` reindex `` de una `` Serie`` o `` DataFrame ``.

### MultiIndex: nombrando los niveles

A veces, es conveniente nombrar los niveles del `` MultiIndex ``.

Esto se puede lograr pasando el argumento `` names `` a cualquiera de los constructores `` MultiIndex `` anteriores, o configurando el atributo `` names `` del índice después de haberlo contruido:

In [41]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [42]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Con conjuntos de datos más complicados, esta puede ser una forma útil de realizar un seguimiento del significado de varios valores de índice.

### MultiIndex para columnas

En un ``DataFrame``, las filas y columnas son completamente simétricas, y así como las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.

Creemos una maqueta de algunos datos médicos:

In [43]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  39.2  32.0  36.6  31.0  36.0
     2      41.0  36.9  17.0  38.5  36.0  37.6
2014 1      35.0  37.9  46.0  37.8  34.0  37.1
     2      42.0  38.2  31.0  38.2  43.0  37.9

Aquí vemos dónde la indexación múltiple para filas y columnas puede ser especialmente útil.

Se trata fundamentalmente de datos de cuatro dimensiones, donde las dimensiones son el sujeto (subject), el tipo de medida (type), el año (year) y el número de visita (visit).

Con esto en su lugar, podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener un `` DataFrame `` completo que contenga solo la información de esa persona:

In [44]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      32.0  36.6
     2      17.0  38.5
2014 1      46.0  37.8
     2      31.0  38.2

In [50]:
health_data.index

MultiIndex([(2013, 1),
            (2013, 2),
            (2014, 1),
            (2014, 2)],
           names=['year', 'visit'])

In [46]:
health_data.loc[(2013, 1), :]

subject  type
Bob      HR      39.0
         Temp    39.2
Guido    HR      32.0
         Temp    36.6
Sue      HR      31.0
         Temp    36.0
Name: (2013, 1), dtype: float64

Para registros complicados que contengan múltiples mediciones etiquetadas en múltiples momentos para muchos sujetos (personas, países, ciudades, etc.), el uso de filas y columnas jerárquicas puede ser extremadamente conveniente.

## Indexing y Slicing en MultiIndex

La indexación y la división en un `` MultiIndex `` está diseñada para ser intuitiva, pensando en los índices como dimensiones adicionales.

Primero, veremos la indexación de `` Series `` indexadas de forma múltiple, y luego los `` DataFrame `` indexados de forma múltiple.

### Seeries indexadas de manera múltiple

Consideremos la `` Serie `` de poblaciones estatales con índices múltiples que vimos anteriormente:

In [51]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Podemos acceder a elementos individuales indexando con varios términos:

In [56]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

El `` MultiIndex `` también admite indexación parcial, o indexar solo uno de los niveles del índice.

El resultado es otra `` Serie `` manteniendo los índices de nivel inferior:

In [59]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

También se puede usar el slicing parcial, siempre que el `` Índice múltiple `` esté ordenado:

In [66]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

Con índices ordenados, la indexación parcial se puede realizar en niveles inferiores pasando un segmento vacío en el primer índice:

In [68]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

También funcionan otros tipos de indexación y selección discutidos anteriomente; por ejemplo, selección basada en máscaras booleanas:

In [69]:
pop > 22000000

state       year
California  2000     True
            2010     True
New York    2000    False
            2010    False
Texas       2000    False
            2010     True
dtype: bool

In [70]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Y también podemos seleccionar una lista de valores del índice:

In [78]:
pop.loc[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [80]:
pop.iloc[[0]]

state       year
California  2000    33871648
dtype: int64

### DataFrames con índices múltiples


Un `` DataFrame `` indexado de forma múltiple se comporta de manera similar. Considerando el ejemplo médico de antes:

In [81]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  39.2  32.0  36.6  31.0  36.0
     2      41.0  36.9  17.0  38.5  36.0  37.6
2014 1      35.0  37.9  46.0  37.8  34.0  37.1
     2      42.0  38.2  31.0  38.2  43.0  37.9

Recuerda que las columnas son prioritarias en un `` DataFrame ``, y la sintaxis utilizada para las `` Series `` con índices múltiples se aplica a las columnas.

Por ejemplo, podemos recuperar los datos de frecuencia cardíaca de Guido con una simple operación:

In [82]:
health_data['Guido', 'HR']

year  visit
2013  1        32.0
      2        17.0
2014  1        46.0
      2        31.0
Name: (Guido, HR), dtype: float64

Del mismo modo, al igual que con el caso de índice único, podemos usar los indexadores `` loc``, ``iloc`` e ``ix``. Por ejemplo:

In [83]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      39.0  39.2
     2      41.0  36.9

Estos indexadores proporcionan una vista similar a una matriz de los datos bidimensionales subyacentes, pero a cada índice individual en "loc" o "iloc" se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [84]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        39.0
      2        41.0
2014  1        35.0
      2        42.0
Name: (Bob, HR), dtype: float64

Trabajar con slices dentro de estas tuplas de índice no es especialmente conveniente; intentar crear un slice dentro de una tupla dará lugar a un error de sintaxis:

In [85]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-85-fb34fa30ac09>, line 1)

Podemos evitar esto construyendo explícitamente el slice deseado usando la función incorporada `` slice () `` de Python, pero una mejor manera en este contexto es usar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación.

Por ejemplo:

In [90]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,39.0,32.0,31.0
2014,1,35.0,46.0,34.0


Hay muchas formas de interactuar con los datos en `` Series `` y `` DataFrame `` indexados de forma múltiple, y como sucede con muchas de las herramientas que hemos visto, la mejor manera de familiarizarse con ellos es probarlas.

## Reorganización de MultiIndex

Una de las claves para trabajar con datos indexados de forma múltiple es saber cómo transformar los datos de forma eficaz.

Hay una serie de operaciones que conservarán toda la información en el conjunto de datos, pero que nos permitirán reorganizar los datos para facilitar los cálculos.

Anteriormente, hemos visto un breve ejemplo de esto en los métodos `` stack () `` y ``unstack ()``, pero hay muchas más formas de controlar con precisión la reordenación de datos entre índices jerárquicos y columnas:

### Índices ordenados y no ordenados

Anteriormente, hemos comentado que muchas de las operaciones de slicing de `` MultiIndex `` fallarán si el índice no está ordenado, lo cual desarrollaremos a continuación.

Comenzaremos creando algunos datos con índices múltiples simples donde los índices no están ordenados lexográficamente:

In [91]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.586177
      2      0.690285
c     1      0.938033
      2      0.742077
b     1      0.440229
      2      0.436724
dtype: float64

Si intentamos tomar un slice parcial de este índice, saltará un error:

In [92]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Aunque no está del todo claro en el mensaje de error, este es el resultado de que el MultiIndex no está ordenado.

Por diversas razones, los slices parciales y otras operaciones similares requieren que los niveles del `` MultiIndex `` estén ordenados (de forma lexográfica).

Pandas proporciona una serie de rutinas de conveniencia para realizar este tipo de clasificación; como son los métodos `` sort_index () `` y `` sortlevel () `` del `` DataFrame ``.

En este caso, usaremos el más simple, `` sort_index () ``:

In [93]:
data = data.sort_index()
data

char  int
a     1      0.586177
      2      0.690285
b     1      0.440229
      2      0.436724
c     1      0.938033
      2      0.742077
dtype: float64

Con el índice ordenado, ya podemos utilizar el slicing:

In [94]:
data['a':'b']

char  int
a     1      0.586177
      2      0.690285
b     1      0.440229
      2      0.436724
dtype: float64

### Stacking y unstacking de índices

Como vimos brevemente antes, es posible convertir un conjunto de datos de un índice múltiple apilado (stacked) a una representación bidimensional simple, especificando opcionalmente el nivel a usar:

In [95]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [65]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [66]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


El opuesto a ``unstack()`` es ``stack()``, el cual aquí puede ser usado para recuperar el DataFrame original:

In [68]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Especificación y reseteo de Index

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de índice en columnas; esto se puede lograr con el método `` reset_index ``.
Llamar a esto en el diccionario de ``population`` dará como resultado un `` DataFrame `` con una columna de ``state`` y ``year`` que contiene la información que anteriormente estaba en el índice.

Para mayor claridad, opcionalmente podemos especificar el nombre de los datos para la representación de la columna:

In [102]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [104]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


A menudo, cuando se trabaja con datos del mundo real, los datos de entrada sin procesar son de este estilo y es útil crear un `` MultiIndex `` a partir de los valores de las columnas.

Esto se puede hacer con el método `` set_index `` del `` DataFrame ``, que devuelve un `` DataFrame `` con un índice múltiple:

In [105]:
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [82]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

En la práctica, veremos que este tipo de reindexación es uno de los patrones más útiles al encontrar conjuntos de datos del mundo real.

## Agregaciones de datos con MultiIndex

Anteriormente, hemos visto que Pandas tiene métodos de agregación de datos integrados, como `` mean () ``, `` sum () `` y `` max () ``.

Para los datos indexados jerárquicamente, estos pueden pasar a un parámetro de `` level `` que controla en qué subconjunto de los datos se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [108]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  39.2  32.0  36.6  31.0  36.0
     2      41.0  36.9  17.0  38.5  36.0  37.6
2014 1      35.0  37.9  46.0  37.8  34.0  37.1
     2      42.0  38.2  31.0  38.2  43.0  37.9

Puede que nos apetezca promediar las mediciones en las dos visitas de cada año. Podemos hacer esto nombrando el nivel de índice que nos gustaría explorar, en este caso el año:

In [109]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     40.0  38.05  24.5  37.55  33.5  36.8
2014     38.5  38.05  38.5  38.00  38.5  37.5

Haciendo uso adicional de la palabra clave ``axis``, también podemos tomar la media entre los niveles en las columnas:

In [85]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,35.666667,37.15
2014,40.666667,36.70


Así, en solamente dos líneas, hemos podido encontrar la frecuencia cardíaca (HR) y la temperatura (Temp) promedio medidas entre todos los sujetos en todas las visitas de cada año.
Esta sintaxis es en realidad un atajo a la funcionalidad `` GroupBy ``, que ya hemos visto.

Si bien este es un ejemplo de prueba, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.

## EXTRA: Panel Data

Pandas tiene algunas otras estructuras de datos fundamentales que aún no hemos discutido, como son los objetos `` pd.Panel `` y `` pd.Panel4D ``.

Estos se pueden considerar, respectivamente, como generalizaciones tridimensionales y tetradimensionales de las estructuras (unidimensionales) de `` Series `` y (bidimensionales) de `` DataFrame ``.
Una vez familiarizados con la indexación y manipulación de datos en una `` Serie `` o `` DataFrame``, el uso de `` Panel `` y `` Panel4D `` es relativamente sencillo. En particular, los indexadores "ix", "loc" y "iloc" discutidos en su día se extienden fácilmente a estos estructuras dimensionales.


No vamos a ver estos tipos en este bootcamp, ya que en la mayoría de los casos la indexación múltiple es una representación más útil y conceptualmente más simple para trabajar con los datos que utilizar dimensiones superiores.
Además, los datos de panel son fundamentalmente una representación de datos densa, mientras que la indexación múltiple es fundamentalmente una representación de datos dispersos.
A medida que aumenta el número de dimensiones, la representación densa puede volverse muy ineficiente para la mayoría de los conjuntos de datos del mundo real.
Sin embargo, para la aplicación especializada ocasional, estas estructuras pueden ser útiles.

## Para probar

Esta vez no vamos a realizar un ejercicio como tal, vamos a dejar volar nuestras dotes de analista. Coge cualquier dataset que hayamos estudiado en clase y crea diferentes tablas en base a las combinaciones de índices que más te interesen para obtener relaciones de datos interesantes.